In [30]:
# import pandas as pd

# HourlyFullCheck
# file_path = '/backupdata/data_EPA/aq_obs/routine/2011/AQS_hourly_data_2011_LatLon.csv'
# complete_count = 0  # 用于统计完整数据的数量
# incomplete_count = 0  # 用于统计不完整数据的数量
# try:
#     df = pd.read_csv(file_path)
#     # 将 dateon 列转换为日期时间类型
#     df['dateon'] = pd.to_datetime(df['dateon'])

#     # 提取日期和小时信息
#     df['date'] = df['dateon'].dt.date
#     df['hour'] = df['dateon'].dt.hour

#     # 按站点 ID 和日期分组
#     grouped = df.groupby(['site_id', 'date'])

#     # 检查每组是否有 24 个小时的数据
#     for (site_id, date), group in grouped:
#         if len(group['hour'].unique()) == 24:
#             complete_count += 1
#         else:
#             incomplete_count += 1

#     print(f"有完整 24 小时数据的站点-日期组合数量: {complete_count}")
#     print(f"数据不完整的站点-日期组合数量: {incomplete_count}")

# except FileNotFoundError:
#     print(f"错误: 文件 {file_path} 未找到。")
# except Exception as e:
#     print(f"发生未知错误: {e}")
# from itertools import product

# #2011年输出
# # 有完整 24 小时数据的站点-日期组合数量: 246920
# # 数据不完整的站点-日期组合数量: 136149

In [31]:
import pandas as pd

# CalMissingHour
def print_unique_sites_count():
    try:
        # 读取输入文件
        input_file_path = '/backupdata/data_EPA/aq_obs/routine/2011/AQS_hourly_data_2011_LatLon.csv'
        input_df = pd.read_csv(input_file_path)
        input_df['dateon'] = pd.to_datetime(input_df['dateon'])

        # 获取所有唯一的站点信息
        all_sites = input_df.groupby('site_id').first()[['Lat', 'Lon']]

        periods = {
            'DJF': pd.date_range('2011-01-01 00:00:00', '2011-02-28 23:00:00', freq='H').union(
                pd.date_range('2011-12-01 00:00:00', '2011-12-31 23:00:00', freq='H')),
            'MAM': pd.date_range('2011-03-01 00:00:00', '2011-05-31 23:00:00', freq='H'),
            'JJA': pd.date_range('2011-06-01 00:00:00', '2011-08-31 23:00:00', freq='H'),
            'SON': pd.date_range('2011-09-01 00:00:00', '2011-11-30 23:00:00', freq='H'),
            'Apr - Sep': pd.date_range('2011-04-01 00:00:00', '2011-09-30 23:00:00', freq='H'),
            'Annual': pd.date_range('2011-01-01 00:00:00', '2011-12-31 23:00:00', freq='H'),
        }

        results = []
        for period_name, period_dates in periods.items():
            all_hours_count = len(period_dates)
            # 按站点分组，使用向量化操作计算每个站点在该周期内的现有小时数
            period_df = input_df[input_df['dateon'].isin(period_dates)]
            if not period_df.empty:
                site_grouped = period_df.groupby('site_id')
                existing_hours = site_grouped['dateon'].nunique()
            else:
                existing_hours = pd.Series([0] * len(all_sites), index=all_sites.index)

            # 计算所有站点的缺失小时数
            missing_hours = all_hours_count - existing_hours.reindex(all_sites.index, fill_value=0)
            # 创建该周期的结果 DataFrame
            period_result = pd.DataFrame({
                'Period': period_name,
                'site_id': all_sites.index,
                'Lat': all_sites['Lat'],
                'Lon': all_sites['Lon'],
                'MissingHours': missing_hours
            })
            results.append(period_result)

            # 打印该周期的唯一站点数量
            unique_sites_count = len(period_result['site_id'].unique())
            print(f"周期 {period_name} 的唯一站点数量: {unique_sites_count}")

        # 合并所有周期的结果
        if results:
            result_df = pd.concat(results, ignore_index=True)
            # 保存结果到输出文件
            output_file_path = '/backupdata/data_EPA/aq_obs/routine/2011/AQS_hourly_data_2011_LatLon_MissingHours.csv'
            result_df.to_csv(output_file_path, index=False)
            print(f"结果已保存到 {output_file_path}")

            # 打印每个周期缺失最大的站点
            for period in result_df['Period'].unique():
                period_subset = result_df[result_df['Period'] == period]
                max_missing = period_subset[period_subset['MissingHours'] == period_subset['MissingHours'].max()]
                print(f"周期 {period} 中缺失小时数最大的站点信息：")
                print(max_missing)
        else:
            print("没有符合条件的数据，未生成结果文件。")

    except FileNotFoundError:
        print(f"错误：未找到输入文件 {input_file_path}。")
    except Exception as e:
        print(f"发生未知错误：{e}")


print_unique_sites_count()

周期 DJF 的唯一站点数量: 1330
周期 MAM 的唯一站点数量: 1330
周期 JJA 的唯一站点数量: 1330
周期 SON 的唯一站点数量: 1330
周期 Apr - Sep 的唯一站点数量: 1330
周期 Annual 的唯一站点数量: 1330
结果已保存到 /backupdata/data_EPA/aq_obs/routine/2011/AQS_hourly_data_2011_LatLon_MissingHours.csv
周期 DJF 中缺失小时数最大的站点信息：
     Period    site_id        Lat         Lon  MissingHours
0       DJF   10030010  30.497478  -87.880258          2160
1       DJF   10331002  34.762619  -87.638097          2160
3       DJF   10510001  32.498567  -86.136587          2160
4       DJF   10550011  33.904039  -86.053867          2160
5       DJF   10690004  31.188933  -85.423094          2160
...     ...        ...        ...         ...           ...
1288    DJF  551250001  46.051900  -89.654000          2160
1289    DJF  551270005  42.580009  -88.499046          2160
1290    DJF  551330027  43.020075  -88.215070          2160
1297    DJF  560111013  44.596900 -104.704800          2160
1302    DJF  560252601  42.860800 -106.235860          2160

[516 rows x 5 columns]
周期 MAM

In [32]:
# import pandas as pd

#Day
# # 加载数据
# file_path = "/backupdata/data_EPA/EQUATES/EQUATES_data/ds.input.aqs.o3.2011.csv"
# data = pd.read_csv(file_path)

# # 将 'Date' 列转换为日期时间格式
# data['Date'] = pd.to_datetime(data['Date'])

# # 定义季度函数
# def get_quarter(month):
#     if month in [12, 1, 2]:
#         return 'DFJ'
#     elif month in [3, 4, 5]:
#         return 'MAM'
#     elif month in [6, 7, 8]:
#         return 'JJA'
#     elif month in [9, 10, 11]:
#         return 'SON'

# # 添加季度列
# data['Quarter'] = data['Date'].dt.month.apply(get_quarter)

# # 生成每个季度的日期范围
# quarters = {
#     'DJF': pd.date_range('2011-01-01', '2011-02-28').union(pd.date_range('2011-12-01', '2011-12-31')),
#     'MAM': pd.date_range('2011-03-01', '2011-05-31'),
#     'JJA': pd.date_range('2011-06-01', '2011-08-31'),
#     'SON': pd.date_range('2011-09-01', '2011-11-30'),
#     'Apr-Sep': pd.date_range('2011-04-01', '2011-09-30'),
#     'Annual': pd.date_range('2011-01-01', '2011-12-31'),
# }

# # 计算缺失日期的数量
# def count_missing_dates(site_data, quarter_dates):
#     site_quarter_data = site_data[site_data['Date'].isin(quarter_dates)]
#     missing_dates = len(quarter_dates) - len(site_quarter_data)
#     return missing_dates

# # 统计每个站点各个季度的缺失天数
# missing_days_per_site = []

# for site in data['Site'].unique():
#     site_data = data[data['Site'] == site]
#     lat = site_data['Lat'].iloc[0]  # 获取站点纬度
#     lon = site_data['Lon'].iloc[0]  # 获取站点经度

#     for quarter, quarter_dates in quarters.items():
#         missing_days = count_missing_dates(site_data, quarter_dates)
#         missing_days_per_site.append([site, lon, lat, missing_days, quarter])

# # 创建 DataFrame
# missing_days_df = pd.DataFrame(missing_days_per_site, columns=['StationID', 'Lon', 'Lat', 'MissingDays', 'Period'])

# # ...existing code...

# # 保存结果到 CSV 文件
# output_file_path = "/backupdata/data_EPA/EQUATES/EQUATES_data/ds.input.aqs.o3.2011_MissingDays.csv"
# missing_days_df.to_csv(output_file_path, index=False)


In [33]:
# import pandas as pd

# # DiffForDayAndHour
# # 定义时间段
# periods = {
#     'DJF': pd.date_range('2011-01-01', '2011-02-28').union(pd.date_range('2011-12-01', '2011-12-31')),
#     'MAM': pd.date_range('2011-03-01', '2011-05-31'),
#     'JJA': pd.date_range('2011-06-01', '2011-08-31'),
#     'SON': pd.date_range('2011-09-01', '2011-11-30'),
#     'Apr-Sep': pd.date_range('2011-04-01', '2011-09-30'),
#     'Annual': pd.date_range('2011-01-01', '2011-12-31'),
# }

# # 读取小时数据表和日数据表
# try:
#     hourly_data = pd.read_csv('/backupdata/data_EPA/aq_obs/routine/2011/AQS_hourly_data_2011_LatLon.csv')
#     daily_data = pd.read_csv('/backupdata/data_EPA/EQUATES/EQUATES_data/ds.input.aqs.o3.2011.csv')

#     # 将小时数据的 'dateon' 列和日数据的 'Date' 列转换为 datetime 类型
#     hourly_data['dateon'] = pd.to_datetime(hourly_data['dateon'])
#     daily_data['Date'] = pd.to_datetime(daily_data['Date'])

#     result = []

#     for period_name, date_range in periods.items():
#         # 筛选出该时间段内的小时数据和日数据
#         hourly_period_data_isin = hourly_data[hourly_data['dateon'].isin(date_range)]
#         daily_period_data_isin = daily_data[daily_data['Date'].isin(date_range)]

#         hourly_period_data_logic = hourly_data[(hourly_data['dateon'] >= date_range.min()) & (hourly_data['dateon'] <= date_range.max())]
#         daily_period_data_logic = daily_data[(daily_data['Date'] >= date_range.min()) & (daily_data['Date'] <= date_range.max())]

#         # 统计每个站点在 isin 筛选后的记录数量
#         hourly_site_counts_isin = hourly_period_data_isin.groupby('site_id').size()
#         daily_site_counts_isin = daily_period_data_isin.groupby('Site').size()

#         # 统计每个站点在逻辑判断筛选后的记录数量
#         hourly_site_counts_logic = hourly_period_data_logic.groupby('site_id').size()
#         daily_site_counts_logic = daily_period_data_logic.groupby('Site').size()

#         print(f"时间段 {period_name}：")
#         print("使用 isin 筛选 - 小时数据各站点记录数量统计：")
#         print(hourly_site_counts_isin)
#         print("使用 isin 筛选 - 日数据各站点记录数量统计：")
#         print(daily_site_counts_isin)

#         print("使用逻辑判断筛选 - 小时数据各站点记录数量统计：")
#         print(hourly_site_counts_logic)
#         print("使用逻辑判断筛选 - 日数据各站点记录数量统计：")
#         print(daily_site_counts_logic)

#         # 找出该时间段内小时数据和日数据中站点 ID 的差异
#         diff_site_ids = set(hourly_period_data_isin['site_id']).symmetric_difference(set(daily_period_data_isin['Site']))

#         # 获取差异站点 ID 的 Lat 和 Lon 信息
#         for site_id in diff_site_ids:
#             site_info = hourly_period_data_isin[hourly_period_data_isin['site_id'] == site_id].head(1)[['site_id', 'Lat', 'Lon']]
#             if not site_info.empty:
#                 site_info['Period'] = period_name
#                 site_info['Flag'] = 2
#             else:
#                 site_info = daily_period_data_isin[daily_period_data_isin['Site'] == site_id].head(1)[['Site', 'Lat', 'Lon']]
#                 site_info.rename(columns={'Site': 'site_id'}, inplace=True)
#                 site_info['Period'] = period_name
#                 site_info['Flag'] = 1
#             result.append(site_info)

#     # 合并结果
#     result_df = pd.concat(result, ignore_index=True)

#     # 输出结果到指定路径
#     output_path = '/backupdata/data_EPA/aq_obs/routine/2011/2011_site_id_difference.csv'
#     result_df.to_csv(output_path, index=False)
#     print(f"结果已保存到 {output_path}")

# except FileNotFoundError:
#     print("文件未找到，请检查文件路径是否正确。")
# except Exception as e:
#     print(f"发生未知错误: {e}")

In [34]:
import pandas as pd

# 定义时间段
periods = {
    'DJF': pd.date_range('2011-01-01', '2011-02-28').union(pd.date_range('2011-12-01', '2011-12-31')),
    'MAM': pd.date_range('2011-03-01', '2011-05-31'),
    'JJA': pd.date_range('2011-06-01', '2011-08-31'),
    'SON': pd.date_range('2011-09-01', '2011-11-30'),
    'Apr-Sep': pd.date_range('2011-04-01', '2011-09-30'),
    'Annual': pd.date_range('2011-01-01', '2011-12-31'),
}

# 读取小时数据表和日数据表
try:
    hourly_data = pd.read_csv('/backupdata/data_EPA/aq_obs/routine/2011/AQS_hourly_data_2011_LatLon.csv')
    daily_data = pd.read_csv('/backupdata/data_EPA/EQUATES/EQUATES_data/ds.input.aqs.o3.2011.csv')

    # 将小时数据的 'dateon' 列和日数据的 'Date' 列转换为 datetime 类型
    hourly_data['dateon'] = pd.to_datetime(hourly_data['dateon'])
    daily_data['Date'] = pd.to_datetime(daily_data['Date'])

    result = []

    for period_name, date_range in periods.items():
        if period_name == 'DJF':
            # 对于 DJF 时间段，分别处理两段日期范围
            first_range = pd.date_range('2011-01-01', '2011-02-28')
            second_range = pd.date_range('2011-12-01', '2011-12-31')
            hourly_period_data = hourly_data[
                ((hourly_data['dateon'] >= first_range.min()) & (hourly_data['dateon'] <= first_range.max())) |
                ((hourly_data['dateon'] >= second_range.min()) & (hourly_data['dateon'] <= second_range.max()))
            ]
            daily_period_data = daily_data[
                ((daily_data['Date'] >= first_range.min()) & (daily_data['Date'] <= first_range.max())) |
                ((daily_data['Date'] >= second_range.min()) & (daily_data['Date'] <= second_range.max()))
            ]
        else:
            # 其他时间段使用常规逻辑判断筛选
            hourly_period_data = hourly_data[(hourly_data['dateon'] >= date_range.min()) & (hourly_data['dateon'] <= date_range.max())]
            daily_period_data = daily_data[(daily_data['Date'] >= date_range.min()) & (daily_data['Date'] <= date_range.max())]

        # 计算各个时间段各自的 Unique 站点个数
        hourly_unique_count = len(set(hourly_period_data['site_id']))
        daily_unique_count = len(set(daily_period_data['Site']))

        print(f"时间段 {period_name}：")
        print(f"小时数据的 Unique 站点个数: {hourly_unique_count}")
        print(f"日数据的 Unique 站点个数: {daily_unique_count}")

        # 找出该时间段内小时数据和日数据中站点 ID 的差异
        diff_site_ids = set(hourly_period_data['site_id']).symmetric_difference(set(daily_period_data['Site']))

        # 获取差异站点 ID 的 Lat 和 Lon 信息
        for site_id in diff_site_ids:
            site_info = hourly_period_data[hourly_period_data['site_id'] == site_id].head(1)[['site_id', 'Lat', 'Lon']]
            if not site_info.empty:
                site_info['Period'] = period_name
                site_info['Flag'] = 2
            else:
                site_info = daily_period_data[daily_period_data['Site'] == site_id].head(1)[['Site', 'Lat', 'Lon']]
                site_info.rename(columns={'Site': 'site_id'}, inplace=True)
                site_info['Period'] = period_name
                site_info['Flag'] = 1
            result.append(site_info)

    # 合并结果
    result_df = pd.concat(result, ignore_index=True)

    # 输出结果到指定路径
    output_path = '/backupdata/data_EPA/aq_obs/routine/2011/2011_site_id_difference.csv'
    result_df.to_csv(output_path, index=False)
    print(f"结果已保存到 {output_path}")

except FileNotFoundError:
    print("文件未找到，请检查文件路径是否正确。")
except Exception as e:
    print(f"发生未知错误: {e}")
    

时间段 DJF：
小时数据的 Unique 站点个数: 813
日数据的 Unique 站点个数: 799
时间段 MAM：
小时数据的 Unique 站点个数: 1301
日数据的 Unique 站点个数: 1266
时间段 JJA：
小时数据的 Unique 站点个数: 1318
日数据的 Unique 站点个数: 1285
时间段 SON：
小时数据的 Unique 站点个数: 1311
日数据的 Unique 站点个数: 1279
时间段 Apr-Sep：
小时数据的 Unique 站点个数: 1322
日数据的 Unique 站点个数: 1290
时间段 Annual：
小时数据的 Unique 站点个数: 1330
日数据的 Unique 站点个数: 1299
结果已保存到 /backupdata/data_EPA/aq_obs/routine/2011/2011_site_id_difference.csv
